In [67]:
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime as dt

import json
import numpy as np
import psycopg2
from zipfile import *
from sql_functions import *

import math


def get_description(bs):
    # find all the descriptions and save them to a list
    lst_name = []
    descriptions = bs.find_all(
        class_='homecard-content__title__HomecardContent___OmV4c homecard-content__title--rebranding-style__HomecardContent___OmV4c')
    # iterate over the list appartement to get the text and strip the strings
    for description in descriptions:
        lst_name.append(
            description.get_text()
                .strip()
        )
    return lst_name


# Creating a function to get all the prices
def get_price(bs):
    # find all the prices and save them to a list
    prices = bs.find_all(class_='price__Price___OmV4c')
    lst_name = []
    # iterate over the prices
    for price in prices:
        lst_name.append(
            price.get_text()
                .strip()
        )
    return lst_name

# Creating a function to get all the housing types


def get_housing(bs):
    bs
    # find all the housing types and save them to a list
    housings = bs.find_all(
        class_='homecard-content__type__HomecardContent___OmV4c homecard-content__type--rebranding-style__HomecardContent___OmV4c')
    lst_name = []
    # iterate over the prices
    for housing in housings:
        lst_name.append(
            housing.get_text()
                .strip()
        )
    return lst_name

# Creating a function to get all the available dates


def get_available(bs):
    # find all the avaiable dates and save them to a list
    availables = bs.find_all(
        class_='homecard-content__available-from__HomecardContent___OmV4c homecard-content__available-from--rebranding-style__HomecardContent___OmV4c')
    lst_name = []
    # iterate over the prices
    for available in availables:
        lst_name.append(
            available.get_text()
                .strip()
        )
    return lst_name

# Creating a function to get all the available dates


def get_prices_month(bs):
    # find all the avaiable dates and save them to a list
    prices_month = bs.find_all(
        class_='price-monthly__Price___OmV4c price-monthly--rebranding-style__Price___OmV4c')
    lst_name = []
    # iterate over the prices
    for price_month in prices_month:
        lst_name.append(
            price_month.get_text()
                .strip()
                .replace('/', '')
        )
    return lst_name

location_dict = {219: 'Lambeth',
                231: 'Hammersmith and Fulham',
                232: 'Kensington and Chelsea',
                233: 'Westminster',
                234: 'Camden',
                235: 'Tower Hamlets',
                236: 'Islington',
                237: 'Hackney',
                241: 'City of London'
}

def page_results(property_type, location):
    page = requests.get(
        'https://www.spotahome.com/de/s/london--uk/for-rent:{}?areaId[]={}'.format(property_type, location))
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # Extracting all the special offers by using the div tag and the class 'special-tags p-2'
    results = bs.find_all('h1', {'class': 'search-title__title'})

    result_text = 0

    # to get only the text from the specials we are iterating over all specials
    for result in results:
        # and than get the text of all spans from the special objects
        result_text = result.find("strong").get_text().strip()

    result_converted = int(result_text)

    page_site = result_converted / 60
    page_site = math.ceil(page_site)

    page_converted = int(page_site)

    df_search = pd.DataFrame()

    begin = 'https://www.spotahome.com/de/s/london--uk/for-rent:{}'.format(property_type)
    end = '?areaId[]={}'.format(location)

    page_number = 1
    page_converted = page_converted + 1

    # for loop to get the page numbers
    for page_number in range(page_converted):
        # sleep timer to reduce the traffic for the server
        #time.sleep(0.5)

        # get the content
        page = requests.get(begin+f'/page:{page_number}'+end)
        html = page.content
        bs = BeautifulSoup(html, 'html.parser')

        # create a pandas dataframe for the names and prices
        spotahome_dict = {
            'housing_type': get_housing(bs),
            'available': get_available(bs),
            'description': get_description(bs),
            'prices': get_price(bs),
            'prices_per': get_prices_month(bs),
            'location': location_dict[location],
            'property_type': property_type
        }
        df_page = pd.DataFrame(data=spotahome_dict)

        # append the temporary dataframe to the dataframe we created earlier outside the for loop
        df_search = df_search.append(df_page)
    return(df_search)


# list with different property types given from the website
property_types = ['studios', 'apartments/bedrooms:1',
                  'apartments/bedrooms:2', 'apartments/bedrooms:3', 'apartments/bedrooms:3more']
locations = [219, 231, 232, 233, 234, 235, 236, 237, 241]

df_complete = pd.DataFrame()
# for loop to get the different property types
for property_type in property_types:
    # for loop to get the different locations
    for location in locations:
        df_complete = df_complete.append(page_results(property_type, location))



/var/folders/qr/fv9yw31x49lgvhl5742z3ds80000gn/T/ipykernel_11153/1657164351.py:159: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df_page)
/var/folders/qr/fv9yw31x49lgvhl5742z3ds80000gn/T/ipykernel_11153/1657164351.py:173: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_complete = df_complete.append(page_results(property_type, location))
/var/folders/qr/fv9yw31x49lgvhl5742z3ds80000gn/T/ipykernel_11153/1657164351.py:159: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df_page)
/var/folders/qr/fv9yw31x49lgvhl5742z3ds80000gn/T/ipykernel_11153/1657164351.py:173: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

In [68]:
display(df_complete)

,housing_type,available,description,prices,prices_per,location,property_type
0,Studio,Verfügbar ab 15 Oktober,"Studio-Apartment zu vermieten in Kensal Green,...",£1452,Monat,Hammersmith and Fulham,studios
1,Studio,Verfügbar ab 10 Oktober,Die Armadale Road Residenz II,£2650,Monat,Hammersmith and Fulham,studios
2,Studio,Verfügbar ab 20 Dezember,Die Armadale Road Residenz III,£2500,Monat,Hammersmith and Fulham,studios
0,Studio,Verfügbar ab 26 September,Studio zu vermieten in einem Wohngemeinschafts...,£2250,Monat,Kensington and Chelsea,studios
1,Studio,Verfügbar ab 26 September,Studio zu vermieten in einem Wohngemeinschafts...,£2300,Monat,Kensington and Chelsea,studios
...,...,...,...,...,...,...,...
3,Wohnung,Verfügbar ab 20 November,4-Zimmer-Wohnung zu vermieten in London,8250-£10950,Monat,Hackney,apartments/bedrooms:3more
4,Wohnung,Verfügbar ab 11 Januar 2023,4-Zimmer-Wohnung zu vermieten in London,9000-£11250,Monat,Hackney,apartments/bedrooms:3more
5,Wohnung,Verfügbar ab 6 Januar 2024,4-Zimmer-Wohnung in Hackney,£2100,Monat,Hackney,apartments/bedrooms:3more
6,Wohnung,Verfügbar ab 4 August 2023,Lagerumbau mit 4 Schlafzimmern zur Miete in Ha...,9000-£11250,Monat,Hackney,apartments/bedrooms:3more


In [69]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844 entries, 0 to 7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   housing_type   844 non-null    object
 1   available      844 non-null    object
 2   description    844 non-null    object
 3   prices         844 non-null    object
 4   prices_per     844 non-null    object
 5   location       844 non-null    object
 6   property_type  844 non-null    object
dtypes: object(7)
memory usage: 52.8+ KB


In [70]:
# schema = 'capstone-jmrs'
# engine = get_engine()

# table_name = 'spotahome'

# if engine!=None:
#     try:
#         df1.to_sql(name=table_name,
#                         con=engine,
#                         if_exists='replace',
#                         schema=schema, 
#                         index=False,
#                         chunksize=5000, 
#                         method='multi')
#         print(f"The {table_name} table was imported successfully.")
    
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [71]:
df_id = pd.DataFrame()

# create global list of targeted neighbourhoods AND match with platform location IDs
neighbourhoods = ['Hammersmith and Fulham', 'Kensington and Chelsea', 'Camden', 'Westminster', 'City of London', 'Hackney', 'Lambeth', 'Tower Hamlets', 'Islington']
spotahome_loc_id = ['231', '232', '234', '233', '241', '237', '219', '235', '236']
zip_iterator = zip(neighbourhoods, spotahome_loc_id)
spotahome_dict_id = dict(zip_iterator)

df_spot_id = pd.DataFrame(data=spotahome_dict_id, index=[0])

df_id = df_id.append(df_spot_id)

#df_studio = pd.DataFrame(data=spotahome_dict)
#df1 = df1.append(df_studio)



/var/folders/qr/fv9yw31x49lgvhl5742z3ds80000gn/T/ipykernel_11153/52047792.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_id = df_id.append(df_spot_id)


## Studio

In [72]:
# begin = 'https://www.spotahome.com/de/s/london--uk/for-rent:studios'
# end = '?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'
# link = begin + end

# #'https://www.spotahome.com/de/s/london--uk/for-rent:studios?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'

# # for loop to get the page numbers
# for _ in range(15):
#    print(_)
#    # sleep timer to reduce the traffic for the server
#    time.sleep(5)
#    # create the lists for the functions
#    housings_lst = []
#    available_lst = []
#    descriptions_lst = []
#    prices_lst = []
#    prices_month_lst =[]
#    # first iteration is the main page
#    if _ == 0:
#        # get the content
#        page = requests.get(link)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_studio = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df1 = df1.append(df_studio)
#    # else-statment for the next pages
#    else:
#        # get the content from the links we create with a f-string an the number we get from the for-loop
#        page = requests.get(begin+f'/page:{_}'+end)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_studio = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df1 = df1.append(df_studio)

In [73]:
# schema = 'muc_analytics_22_1'
# engine = get_engine()

# table_name = 'spotahome'

# if engine!=None:
#     try:
#         df1.to_sql(name=table_name,
#                         con=engine,
#                         if_exists='replace',
#                         schema=schema, 
#                         index=False,
#                         chunksize=5000, 
#                         method='multi')
#         print(f"The {table_name} table was imported successfully.")
    
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [74]:
# df1.info()

In [75]:
# df1.head()

## Appartement 1 Bedroom

In [76]:
# begin = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:1'
# end = '?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'
# link = begin + end

# #link = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:1?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'

# # for loop to get the page numbers
# for _ in range(22):
#    # sleep timer to reduce the traffic for the server
#    time.sleep(3)
#    # create the lists for the functions
#    housings_lst = []
#    available_lst = []
#    descriptions_lst = []
#    prices_lst = []
#    prices_month_lst =[]
#    # first iteration is the main page
#    if _ == 0:
#        # get the content
#        page = requests.get(link)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_1 = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df2 = df2.append(df_app_1)
#    # else-statment for the next pages
#    else:
#        # get the content from the links we create with a f-string an the number we get from the for-loop
#        page = requests.get(begin+f'/page:{_}'+end)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_1 = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df2 = df2.append(df_app_1)

In [77]:
# df2.info()

In [78]:
# df2.head()

## Appartement 2 bedrooms

In [79]:
# begin = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:2'
# end = '?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'
# link = begin + end

# #link = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:2?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'

# # for loop to get the page numbers
# for _ in range(22):
#    # sleep timer to reduce the traffic for the server
#    time.sleep(3)
#    # create the lists for the functions
#    housings_lst = []
#    available_lst = []
#    descriptions_lst = []
#    prices_lst = []
#    prices_month_lst =[]
#    # first iteration is the main page
#    if _ == 0:
#        # get the content
#        page = requests.get(link)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_2 = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df3 = df3.append(df_app_2)
#    # else-statment for the next pages
#    else:
#        # get the content from the links we create with a f-string an the number we get from the for-loop
#        page = requests.get(begin+f'/page:{_}'+end)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_2 = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df3 = df3.append(df_app_2)

In [80]:
# df3.info()

In [81]:
# df3.head()

## Appartement 3 bedrooms

In [82]:
# begin = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:3'
# end = '?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'
# link = begin + end

# #link = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:3?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'

# # for loop to get the page numbers
# for _ in range(22):
#    # sleep timer to reduce the traffic for the server
#    time.sleep(3)
#    # create the lists for the functions
#    housings_lst = []
#    available_lst = []
#    descriptions_lst = []
#    prices_lst = []
#    prices_month_lst =[]
#    # first iteration is the main page
#    if _ == 0:
#        # get the content
#        page = requests.get(link)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_3 = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df4 = df4.append(df_app_3)
#    # else-statment for the next pages
#    else:
#        # get the content from the links we create with a f-string an the number we get from the for-loop
#        page = requests.get(begin+f'/page:{_}'+end)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_3 = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df4 = df4.append(df_app_3)

In [83]:
# df4.info()

In [84]:
# df4.head()

## Appartement 3+ bedrooms

In [85]:
# begin = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:3more'
# end = '?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'
# link = begin + end

# #link = 'https://www.spotahome.com/de/s/london--uk/for-rent:apartments/bedrooms:3more?areaId[]=219&areaId[]=231&areaId[]=232&areaId[]=233&areaId[]=234&areaId[]=235&areaId[]=236&areaId[]=237&areaId[]=241'

# # for loop to get the page numbers
# for _ in range(22):
#    # sleep timer to reduce the traffic for the server
#    time.sleep(3)
#    # create the lists for the functions
#    housings_lst = []
#    available_lst = []
#    descriptions_lst = []
#    prices_lst = []
#    prices_month_lst =[]
#    # first iteration is the main page
#    if _ == 0:
#        # get the content
#        page = requests.get(link)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_3_plus = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df5 = df5.append(df_app_3_plus)
#    # else-statment for the next pages
#    else:
#        # get the content from the links we create with a f-string an the number we get from the for-loop
#        page = requests.get(begin+f'/page:{_}'+end)
#        html = page.content
#        bs = BeautifulSoup(html, 'html.parser')
#        # call the functions to get the information
#        get_description(descriptions_lst)
#        get_price(prices_lst)
#        get_housing(housings_lst)
#        get_available(available_lst)
#        get_prices_month(prices_month_lst)
#        # create a pandas dataframe for the names and prices
#        spotahome_dict = {
#            'housing_type': housings_lst,
#            'available': available_lst,
#            'description': descriptions_lst,
#            'prices': prices_lst,
#            'prices_per': prices_month_lst
#        }
#        df_app_3_plus = pd.DataFrame(data=spotahome_dict)
#        # append the temporary dataframe to the dataframe we created earlier outside the for loop
#        df5 = df5.append(df_app_3_plus)

In [86]:
# df5.info()

In [87]:
# df5.head()

## Merge Dataframes

In [88]:
#df = pd.merge(df1, df2, df3, df4, df5, how = 'left', on='price_per_')

#df_merge1 = pd.merge(df1, df2, how = 'left', on='prices_per')

df_combined = pd.concat([df1, df2, df3, df4, df5], axis=0, ignore_index=True)

NameError: name 'df1' is not defined

In [ ]:
# schema = 'capstone-jmrs'
# engine = get_engine()

# table_name = 'spotahome'

# if engine!=None:
#     try:
#         df.to_sql(name=table_name,
#                         con=engine,
#                         if_exists='replace',
#                         schema=schema, 
#                         index=False,
#                         chunksize=5000, 
#                         method='multi')
#         print(f"The {table_name} table was imported successfully.")
    
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [ ]:
# with open('Untitled-2.json') as fp:
#     data = json.load(fp)
#     print(data)